In [ ]:
%matplotlib inline


# Harmonic-percussive source separation

This notebook illustrates how to separate an audio signal into
its harmonic and percussive components.

We'll compare the original median-filtering based approach of
`Fitzgerald, 2010 <http://arrow.dit.ie/cgi/viewcontent.cgi?article=1078&context=argcon>`_
and its margin-based extension due to `Dreidger, Mueller and Disch, 2014
<http://www.terasoft.com.tw/conf/ismir2014/proceedings/T110_127_Paper.pdf>`_.


In [ ]:
# ran into this issue
# https://github.com/librosa/librosa/issues/219
# if using mac/linux I had to install ffmpeg to parse the audio. If using windows you might need to do something else. Librosa uses a 3rd party app to load audio files, and it doesn't tell you how to fix errors if there are any with loading 
# the audio files. 

import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display
import sklearn

import numpy as np
from collections import Counter, defaultdict
import time
import math
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import xml.dom.minidom
import os
from os.path import exists
from collections import defaultdict

## Below is the example code on how to utilize Librosa with harmonic-percussive source seperation.

Load an example clip with harmonics and percussives



In [ ]:
y, sr = librosa.load('chocomint.flac', duration=5, offset=10)

Compute the short-time Fourier transform of y



In [ ]:
D = librosa.stft(y)

Decompose D into harmonic and percussive components

$D = D_\text{harmonic} + D_\text{percussive}$



In [ ]:
D_harmonic, D_percussive = librosa.decompose.hpss(D)

We can plot the two components along with the original spectrogram



In [ ]:
# Pre-compute a global reference power from the input spectrum
rp = np.max(np.abs(D))

fig, ax = plt.subplots(nrows=3, sharex=True, sharey=True)

img = librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0])
ax[0].set(title='Full spectrogram')
ax[0].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1])
ax[1].set(title='Harmonic spectrogram')
ax[1].label_outer()

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2])
ax[2].set(title='Percussive spectrogram')
fig.colorbar(img, ax=ax)

The default HPSS above assigns energy to each time-frequency bin according to
whether a horizontal (harmonic) or vertical (percussive) filter responds higher
at that position.

This assumes that all energy belongs to either a harmonic or percussive source,
but does not handle "noise" well.  Noise energy ends up getting spread between
D_harmonic and D_percussive.

If we instead require that the horizontal filter responds more than the vertical
filter *by at least some margin*, and vice versa, then noise can be removed
from both components.

Note: the default (above) corresponds to margin=1



In [ ]:
# Let's compute separations for a few different margins and compare the results below
D_harmonic2, D_percussive2 = librosa.decompose.hpss(D, margin=2)
D_harmonic4, D_percussive4 = librosa.decompose.hpss(D, margin=4)
D_harmonic8, D_percussive8 = librosa.decompose.hpss(D, margin=8)
D_harmonic16, D_percussive16 = librosa.decompose.hpss(D, margin=16)

In the plots below, note that vibrato has been suppressed from the harmonic
components, and vocals have been suppressed in the percussive components.



In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=2, sharex=True, sharey=True, figsize=(10, 10))
librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0, 0])
ax[0, 0].set(title='Harmonic')

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[0, 1])
ax[0, 1].set(title='Percussive')
print(D_percussive)

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic2), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive2), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[1, 1])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic4), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive4), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[2, 1])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic8), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[3, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive8), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[3, 1])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_harmonic16), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[4, 0])

librosa.display.specshow(librosa.amplitude_to_db(np.abs(D_percussive16), ref=rp),
                         y_axis='log', x_axis='time', ax=ax[4, 1])

for i in range(5):
    ax[i, 0].set(ylabel='margin={:d}'.format(2**i))
    ax[i, 0].label_outer()
    ax[i, 1].label_outer()

## Below is where code starts for 0x40 hues to parse respacks and audio.

In [ ]:
# parse for the song names and folders

In [ ]:
respack_folders = []
for directory, sub_directories, files in os.walk('./Respacks/'):
    for folder in sub_directories:
        if folder != 'Songs' and folder != 'characters' and folder != 'Images':
            respack_folders.append(folder)
print(respack_folders)

In [ ]:
# Parse and get the rhythm and build up rhythm text from XML files and put them into hashmaps.

In [ ]:
rhythm_map = defaultdict(str)
build_up_rhythm_map = defaultdict(str)

for folder in respack_folders:
    path_name = './Respacks/' + folder + '/songs.xml'
    print(path_name)
    if exists(path_name):
        song_xml = xml.dom.minidom.parse(path_name)
        songs = song_xml.getElementsByTagName('song')
        for song in songs:
            # parsing rhythms
            curr_song_name = song.getAttribute('name')
            if len(song.getElementsByTagName('rhythm')) > 0:
                rhythm_map[curr_song_name] = song.getElementsByTagName('rhythm')[0].firstChild.nodeValue
            if len(song.getElementsByTagName('buildup')) > 0:
                build_up_name = song.getElementsByTagName('buildup')[0].firstChild.nodeValue
                if len(song.getElementsByTagName('buildupRhythm')) > 0:
                    build_up_rhythm_map[build_up_name] = song.getElementsByTagName('buildupRhythm')[0].firstChild.nodeValue
            
    else:
        print('bad path name ' + str(path_name))

print(rhythm_map.items())
print(build_up_rhythm_map.items())

In [ ]:
   
def parseSong(y, sr):
    T = 30.0    # seconds
    t = np.linspace(0, T, int(T*sr), endpoint=False) # time variable
    x = 0.5*np.sin(2*np.pi*220*t)# pure sine wave at 220 Hz
    D = librosa.stft(y)
    ret_dict = dict()
    ret_dict['D_harmonic4'], ret_dict['D_percussive4'] = librosa.decompose.hpss(D, margin=4)
    ret_dict['D_harmonic16'], ret_dict['D_percussive16'] = librosa.decompose.hpss(D, margin=16)
    # ret_dict['spectral_centroids'] = librosa.feature.spectral_centroid(x, sr=sr)[0]
    # ret_dict['spectral_rolloff'] = librosa.feature.spectral_rolloff(x+0.01, sr=sr)[0]
    
    
    #spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr)[0]
    #spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=3)[0]
    #spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(x+0.01, sr=sr, p=4)[0]
    #ret_dict['spectral_bandwidth_2'] = spectral_bandwidth_2
    #ret_dict['spectral_bandwidth_3'] = spectral_bandwidth_3
    #ret_dict['spectral_bandwidth_4'] = spectral_bandwidth_4
    return ret_dict

# dict(audio_name -> dict(features -> values))
audio_rhythm_feature_map = defaultdict(dict)
audio_build_up_rhythm_feature_map = defaultdict(dict)

for folder in respack_folders:
    path_name = './Respacks/' + folder + '/songs.xml'
    print(path_name)
    if exists(path_name):
        song_xml = xml.dom.minidom.parse(path_name)
        songs = song_xml.getElementsByTagName('song')
        for song in songs:
            # parsing rhythms
            curr_song_name = song.getAttribute('name')
            song_path_name = None
            is_rhythm = False
            is_build_up = False
            if len(song.getElementsByTagName('rhythm')) > 0:
                is_rhythm = True
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + curr_song_name + '.mp3'
            
            if len(song.getElementsByTagName('buildup')) > 0:
                is_build_up - True
                build_up_name = song.getElementsByTagName('buildup')[0].firstChild.nodeValue
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + build_up_name + '.mp3'
                    
            # load the song
            if exists(song_path_name):
                print('parsing ' + str(song_path_name))
                try:
                    y, sr = librosa.load(song_path_name, duration=5, offset=10)
                    if is_rhythm:
                        audio_rhythm_feature_map[curr_song_name] = parseSong(y,sr)
                    elif is_build_up:
                        audio_build_up_rhythm_feature_map[build_up_name] = parseSong(y,sr)
                except Exception as e:
                    print('exception occurred')
                    print(e)
            else:
                print('file name doesn\'t exist ' + rhythm_song_path_name)
    else:
        print('bad path name ' + str(path_name))
        

In [ ]:
# This is just to print out the parsed data to take a look. Modify total_files_to_print to determine how many files to print. 

In [ ]:
index = 0
total_files_to_print = 10
for folder in respack_folders:
    if index == total_files_to_print:
        break
    path_name = './Respacks/' + folder + '/songs.xml'
    print(path_name)
    if exists(path_name):
        song_xml = xml.dom.minidom.parse(path_name)
        songs = song_xml.getElementsByTagName('song')
        for song in songs:
            index += 1
            if index == total_files_to_print:
                break
            # parsing rhythms
            curr_song_name = song.getAttribute('name')
            song_path_name = None
            is_rhythm = False
            is_build_up = False
            if len(song.getElementsByTagName('rhythm')) > 0:
                is_rhythm = True
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + curr_song_name + '.mp3'
            
            if len(song.getElementsByTagName('buildup')) > 0:
                is_build_up - True
                build_up_name = song.getElementsByTagName('buildup')[0].firstChild.nodeValue
                song_path_name = path_name = './Respacks/' + folder +'/Songs/' + build_up_name + '.mp3'
                    
            # load the song
            if exists(song_path_name):
                print('outputting features for ' + str(song_path_name))
                if is_rhythm:
                    for key in audio_rhythm_feature_map[curr_song_name].keys():
                        print(key)
                        print(len(audio_rhythm_feature_map[curr_song_name][key]))
                elif is_build_up:
                    print(audio_build_up_rhythm_feature_map[build_up_name])
            else:
                print('file name doesn\'t exist ' + rhythm_song_path_name)
    else:
        print('bad path name ' + str(path_name))
        

In [ ]:
# Data clean up. Need to post-process the data for the models .

In [88]:
# features = audio features
# labels = rhythm/buildup
features = []
labels = []
for song_name in rhythm_map.keys():
    if song_name in audio_rhythm_feature_map:
        # if no features were extracted for some reason, then we can't process it.
        if len(audio_rhythm_feature_map[song_name]) != 0:
            list_to_add = list()
            for feature in audio_rhythm_feature_map[song_name]:
                audio_features_flattened = np.array(audio_rhythm_feature_map[song_name][feature]).flatten()
                for x in audio_features_flattened:
                    complex_to_real = x.real + x.imag
                    list_to_add.append(complex_to_real)
            if len(list_to_add) == 885600:
                labels.append(rhythm_map[song_name])
                features.append(list_to_add)
print(len(labels))
print(len(features))
# print(total/130)
# audio_rhythm_features = audio_rhythm_feature_map.values()
# audio_rhythm_labels = rhythm_map.values()

51
51


In [89]:
print(len(labels))
print(len(features))
for feature_index in range(len(features)):
    print(len(features[feature_index]))


51
51
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600
885600


In [93]:
post_processed_labels = []
post_processed_features = []
# for index in range(len(labels)):
for index in range(10):
    for beat_index in range(len(labels[index])):
        list_to_add = []
        beat_length = len(features[index])//len(labels[index])
        for feature_index in range(beat_index*beat_length, (beat_index+1)*beat_length):
            value = features[index][feature_index]
            list_to_add.append(value)
        if len(list_to_add) == 1581:
            post_processed_labels.append(labels[index][beat_index])
            post_processed_features.append(list_to_add)
            
    #print(len(post_processed_labels[index]))
    #print(len(post_processed_features[index]))
print(len(post_processed_labels))
print(len(post_processed_features))

560
560


In [94]:
## Defining a model and attempting to build a model utilizing the cleaned data.
print(len(post_processed_labels))
print(len(post_processed_features))
for feature_index in range(len(post_processed_features)):
    print(len(post_processed_features[feature_index]))


560
560
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
1581
15

In [95]:
scaler = StandardScaler()

In [96]:
scaler.fit(post_processed_features)
X_train = scaler.transform(post_processed_features)
# print(X_train[0:10])

In [97]:
# define the model
model = RandomForestClassifier()
# evaluate the model
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, features, labels, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

ValueError: n_splits=2 cannot be greater than the number of members in each class.

In [99]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, post_processed_labels)

/Users/frankhu/opt/anaconda3/envs/ai_env/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(5, 2), random_state=1,
              solver='lbfgs')

In [ ]:
## Parse a single song for output testing.

In [126]:
# dict(audio_name -> dict(features -> values))
path_name = 'chocomint.flac'
print(path_name)
if exists(path_name):
    print('parsing ' + str(path_name))
    try:
        y, sr = librosa.load(path_name, duration=100, offset=10)
        single_song = parseSong(y,sr)
    except Exception as e:
        print('exception occurred')
        print(e)
else:
    print('bad path name ' + str(path_name))

chocomint.flac
parsing chocomint.flac


In [127]:
flattened_feature_input = []
for feature in single_song:
    flattened_feature_input.extend(single_song[feature])
single_feature_input = []
flattened_feature_input = np.array(flattened_feature_input).flatten()
for index in range(0, len(flattened_feature_input), 1581):
    single_feature_input.append([])
    for x in flattened_feature_input[index:index+1581]:
        # print(x)
        complex_to_real = x.real + x.imag
        single_feature_input[-1].append(complex_to_real)
    # print(len(single_feature_input[-1]))
single_feature_input.pop()
'''
for index in range(len(single_feature_input)):
    model.predict(np.array([single_feature_input[index][:2]]))
'''
single_feature_normalized = scaler.transform(single_feature_input)
res = clf.predict(np.array(single_feature_normalized))
for index in range(len(res)):
    print(res[index],end='')

::.x.:..:.x:::.::.xx..x:.:::.::.:::.:.:::.:.x.:.x..:::xx.:::.:.:.::::.::...:::..:.::::::::.:..:.::x...:::..xx.:x:....:::...:......x..:.....:.:.x:.x..::.:x::::::.::.::.::..::xx:::x.:....xx.....:.xx.::x...::.x.:.x::::x:.:x..xx::.::..:::x.:::x::::xx..x:.::xx:.:...::x:::::.::..::....:xx.:.:.xx.x..::.::::::.:x.::.x::.::.:::::.:..x....::::.::....x.....:x.::x.::::x:x:.:.:.x::x.x::x.x.::xx..::..x.:::.:.:xx:::..x.:....:x:..:...:x:..::x......:xx:...:.x.....x..x...x..:...::::.::::...::::xxx.....::xxx.::...:..:.:...x..:....x::.x.:x:..::.x:x....:x..:....x..:.x:....x.....x..x.::.xxx.:x..x...::x.:.::x.:::::..:x..x:x..:.xxx..:.:::.x..::.....x.:.:.::x::::::.....:..x...::...::...::x.x..:.x:.x:.:..x........xxx...:....:..x:...:.x::...x.xx::...x:..:....:....:x.x.:xx::....x:..:x...xx:::::x...x:..:....x..:.:...:.:..:.:x.x..:...:........:xx..x.......x::.x..x:.xx:..::..x:xxx:..x.x:.:.x.:.:x:..x:..:..x..xxx......x....x..x.:..:x.x:::..:x..:....:x::x.:.:.:x.:.x....x...x....:.o:xx:.....:.:..x:.....x.....:.x...:.:.

In [ ]:
print(single_feature_input[0:10])